<a href="https://colab.research.google.com/github/juliuserbach/Semantic-Features/blob/master/Semantic_Features_3DVision_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Our mapping pipeline is of the following structure:

1.   Detection of objects of certain object classes (e.g. traffic sign). Output: object bounding boxes
2.   Triangulation of objects. Output: object position relative to pose
3.   Creating map of objects. (And refining with filter, BA, etc.) Output: List of objects and their corresponding positions
4.   Visualizing map.





We start by loading the benchmark dataset. The Cityscapes dataset is used. 

Scripts for analyzing the dataset can be found here: https://github.com/mcordts/cityscapesScripts

How to download the zip files directly: https://towardsdatascience.com/download-city-scapes-dataset-with-script-3061f87b20d7



Download CityScape files: 

*   leftImg8bit_trainextra.zip
*   disparity_trainextra.zip
*   camera_trainextra.zip
*   vehicle_trainextra.zip

The files are unzipped into data/... respectively.









In [0]:
# Download and unzip (run only once!).
trajectoryname = "schweinfurt"

# Remove data directory if it is already loaded (if needed).
#!rm -r data
# Login.
!wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=ftaubner@ethz.ch&password=semantic_dudes&submit=Login' https://www.cityscapes-dataset.com/login/
# Get left camera images. I put it on OneDrive so that only Schweinfurt has to be downloaded.
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199734&authkey=AC5K24PFcrSPFl4" -O leftImg8bit_trainextra.zip
# Alternatively, it can be downloaded from CityScapes website directly:
#   (!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=4)
# Extract.
!mkdir data
!unzip leftImg8bit_trainextra.zip 'leftImg8bit/train_extra/schweinfurt/*' -d data
# And delete.
!rm leftImg8bit_trainextra.zip

# Get disparity maps.
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199735&authkey=AEwGRlH_TySnyPM" -O disparity_trainextra.zip
# Original file:
#   (!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=22)
!unzip disparity_trainextra.zip 'disparity/train_extra/schweinfurt/*' -d data
!rm disparity_trainextra.zip
# Get camera intrinsics. 
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=9
!unzip camera_trainextra.zip 'camera/train_extra/schweinfurt/*' -d data
!rm camera_trainextra.zip
# Get vehicle odometry.
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=11
!unzip vehicle_trainextra.zip 'vehicle/train_extra/schweinfurt/*' -d data
!rm vehicle_trainextra.zip

We start with the detection of objects of interest. Faster R-CNN is used for this task. (Julius)

In [0]:
# install dependencies: (use cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__
!gcc --version
# opencv is pre-installed on colab

In [0]:
# install detectron2:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html

In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

Loading an on COCO pre-trained Faster R-CNN 


In [0]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
predictor = DefaultPredictor(cfg)
im = cv2.imread("Test.jpeg")
plt.imshow(im)
outputs = predictor(im)

In [0]:
#visualize results
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.imshow(v.get_image()[:, :, :])

Installing Seamless Scene Segmentation from https://github.com/mapillary/seamseg
For Seamless Scene Segmentation a pre-trained version on the Mapillary Dataset exists. Dataloaders for Cityscapes seem to exist, too.

It is a network for panoptic segmentation basen on Mask-R-CNN. Alternatively the panoptic variant in the detectron2 reopisitory can also be trained on Cityscapes or Mapillary. Dataloaders seem to exist for both Datasets. 

In [0]:
#@title
!pip install git+https://github.com/mapillary/seamseg.git
!pip install wget

url = 'https://drive.google.com/file/d/1ULhd_CZ24L8FnI9lZ2H6Xuf03n6NA_-Y/view'
wget.download(url)


These functions convert a difference in coordinates (lat., long.) to a difference in metric frame (x, y) and vice versa. Both conversions are dependent on the current latitude.
Under the assumption that we are in Europe, east of 0°!

Approximative conversions:

* Latitude: 1 deg = 110.574 km
* Longitude: 1 deg = 111.320*cos(latitude) km


In [0]:
# Calculates the difference between two coordinates in meters (x to east, y to north)
def coord_diff_to_metric_diff(d_lat, d_long, lat):
  d_y = d_lat * 110574
  d_x = d_long * 111320 * np.cos(np.radians(lat))
  return d_x, d_y

def metric_diff_to_coord_diff(d_x, d_y, lat):
  d_lat = d_y / 110574
  d_long = d_x / 111320 / np.cos(np.radians(lat))
  return d_lat, d_long